# OVERVIEW

(Add some abstract-style text here about the overall results)


# ANALYSES

## Population Genetics

## Phylogeny

## Selection

Outliers were identified using two different programs: BayeScan 2.1 and PCAdapt 4.3.3. This was the version of BayeScan used in the paper, however PCAdapt 4.0.3 was used in the paper. Between possible errors in the linux download of 4.0.3 and the availability of 4.3.3 on bioconda, we opted to use 4.3.3.

### PCAdapt
PCAdapt was very straightforward between the documentation in the methods and the R script on github. When our .vcf file was plugged into the PCAdapt R code, it produced 5,654 SNPs under selection, compared to 4,987 determined in the paper. Because we removed deep sequenced file from our dataset, PCAdapt may have identified SNP outliers that would not have been identified if that deep sequenced sample was in the analysis. However, the SNPs under selection reported by PCAdapt were relatively similar despite the differences in processing that may have arisen between the two analyses.

### BayeScan
The outlier analysis using BayeScan proved to be more difficult. Although information was provided in the paper (version numbers, default settings) and the code was provided for the actual analysis, there was little information on how to get the input files for BayeScan. The author used PGDSpider to create a BayeScan file from the .vcf produced in the identifying SNPs step, but that was all the information provided. There was nothing on how to use PGDSpider, whether the GUI or command line version was used, and what files were used to take population information (host, location, and the interaction between the two) into account when creating these bayescan input files. After using PGDSpider GUI, PGDSpider on the command line, and an R package (radiator) all without success, I used a perl code from github (scripts/bayescan/vcf2bayescan.pl) to create the BayeScan input file. This correctly identified population information and the structure of the file looked as it should. Because of this (and that it worked with BayeScan) I assume that the file was very similar to that which was created for the paper by PGDSpider.

Once the input files were created, I ran BayeScan three separate times, once where information on the host species was included, once where the location of the sample was included, and once where the interaction of the two was included. Because there was not information on how the interaction was identified, I put simply put host species and location together into one category, creating 11 different 'populations.'

My BayeScan output identified 386 outliers when accounting for host, 33 when accounting for location, and 273 when accounting for the interaction of host and location. Between our two analyses, 360 SNPs overlapped between them. All of these outputs were slightly higher than the results of the paper, which had 217 outliers identified when accounting for host, 5 for location, 197 for the interaction, and 339 SNPs overlapping between BayeScan and PCAdapt. Again, I suspect this is the result of missing SNPs present in the deeply sequenced sample. In addition, I cannot confirm that there is no difference in the way the author created her BayeScan files vs. how I created them, possibly causing some differences.